In [1]:
#import statements
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans
from sklearn.preprocessing import MinMaxScaler, LabelEncoder

In [2]:
#mount drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
df = pd.read_csv('/content/drive/MyDrive/Datathon 2024/Chevron/Train/training.csv')
df

,Unnamed: 0,surface_x,surface_y,bh_x,bh_y,standardized_operator_name,gross_perforated_length,number_of_stages,total_proppant,total_fluid,...,relative_well_position,batch_frac_classification,well_family_relationship,frac_type,frac_seasoning,horizontal_midpoint_x,horizontal_midpoint_y,horizontal_toe_x,horizontal_toe_y,OilPeakRate
0,0,1383493.751,717329.2368,1382854.564,712767.3700,1121,3963.582677,NaN,76000.0000,NaN,...,Standalone Well,Unknown,Standalone Well,Primary Frac,NaN,1383030.678,714796.3610,1382668.912,712798.4321,46.623023
1,1,1380344.035,711916.2732,1379738.429,707860.5643,1022,3179.133858,NaN,113000.0000,5.610964e+05,...,Outer Well,Unknown,Infill Child Well,Primary Frac,NaN,1379965.126,709541.7145,1379630.025,707907.6708,59.750009
2,2,1379915.895,717845.0063,1379274.644,713471.9952,1304,3810.039370,12.0,286571.4286,1.621424e+06,...,Standalone Well,Non-Batch Frac,Standalone Well,Primary Frac,92.0,1379357.250,715381.2327,1378960.372,713530.4490,10.785716
3,3,1383567.443,706640.2010,1382891.477,702068.3838,1022,3723.425197,11.0,106878.9286,4.377134e+05,...,Standalone Well,Unknown,Standalone Well,Primary Frac,NaN,1383099.610,704058.8508,1382727.299,702098.4882,123.797638
4,4,1378962.585,702623.9787,1379643.546,707204.9131,1022,3811.023622,NaN,112789.7143,6.229802e+05,...,Outer Well,Unknown,Infill Child Well,Primary Frac,NaN,1379546.414,705150.7519,1379987.487,707123.7258,102.309540
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29687,30087,1441088.593,978921.8705,1432758.032,979984.5869,1050,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,Primary Frac,NaN,NaN,NaN,NaN,NaN,NaN
29688,30088,1513299.189,976157.4486,1519890.072,975148.4820,1159,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,Primary Frac,NaN,NaN,NaN,NaN,NaN,NaN
29689,30089,1458652.142,972862.2346,1453129.889,973195.0998,1050,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,Primary Frac,NaN,NaN,NaN,NaN,NaN,NaN
29690,30090,1463417.436,972485.8276,1468807.765,971564.9743,1153,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,Primary Frac,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
#remove irrelavant columns based on correlation and quantum of null values
def remove_columns(df):
  columns_to_remove = ['Unnamed: 0', 'bh_x', 'bh_y', 'standardized_operator_name',
                       'number_of_stages', 'total_proppant', 'average_stage_length',
                       'average_proppant_per_stage', 'average_frac_fluid_per_stage',
                       'pad_id',  'horizontal_midpoint_x', 'horizontal_midpoint_y',
                       'horizontal_toe_x', 'horizontal_toe_y', 'frac_type']
  return df.drop(columns_to_remove, axis=1)
df1 = remove_columns(df)

In [5]:
#remove rows where OilPeaks is Nan
def remove_target_nan(df):
  return df.loc[df['OilPeakRate'].isnull() == False]

df2 = remove_target_nan(df1)

In [6]:
#remove outliers
def clean_data(df):
  row1 = df[(df["proppant_intensity"]>4000)].index
  df.drop(row1, axis=0, inplace=True)
  row2 = df[(df["frac_fluid_to_proppant_ratio"]==0)].index
  df.drop(row2, axis=0, inplace=True)
  row3 = df[(df["proppant_to_frac_fluid_ratio"]==0)].index
  df.drop(row3, axis=0, inplace=True)
  row4 = df[(df["proppant_to_frac_fluid_ratio"]>100)].index
  df.drop(row4, axis=0, inplace=True)
  return df

df3 = clean_data(df2)

<ipython-input-6-ffc17ec79752>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(row1, axis=0, inplace=True)
<ipython-input-6-ffc17ec79752>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(row2, axis=0, inplace=True)
<ipython-input-6-ffc17ec79752>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(row3, axis=0, inplace=True)
<ipython-input-6-ffc17ec79752>:10: SettingWithCopyWarning: 
A value is trying to be set on 

In [7]:
#calculating remaining null value
df3.isna().sum()/len(df3[['OilPeakRate']])


surface_x                       0.000000
surface_y                       0.000000
gross_perforated_length         0.008185
total_fluid                     0.074600
true_vertical_depth             0.005440
ffs_frac_type                   0.258768
proppant_intensity              0.074289
frac_fluid_intensity            0.076931
proppant_to_frac_fluid_ratio    0.093509
frac_fluid_to_proppant_ratio    0.093509
bin_lateral_length              0.008185
relative_well_position          0.000000
batch_frac_classification       0.000000
well_family_relationship        0.000000
frac_seasoning                  0.250065
OilPeakRate                     0.000000
dtype: float64

In [8]:
#Removing null value for numeric values
def fill_nan_1(df):
  #Fill in NaN Values with mean after checking for outliers and spread of data - bin_lateral_length, gross_perforated_length
  df['bin_lateral_length'].fillna(value=df.bin_lateral_length.mean(), inplace=True)
  df['gross_perforated_length'].fillna(value=df.gross_perforated_length.mean(), inplace=True)
  df['proppant_intensity'].fillna(value=df.proppant_intensity.mean(), inplace=True)
  df['total_fluid'].fillna(value=df.total_fluid.mean(), inplace=True)
  df['frac_fluid_to_proppant_ratio'].fillna(value=df.frac_fluid_to_proppant_ratio.mean(), inplace=True)
  df['proppant_to_frac_fluid_ratio'].fillna(value=df.proppant_to_frac_fluid_ratio.mean(), inplace=True)
  df['true_vertical_depth'].fillna(value=df.true_vertical_depth.mean(), inplace=True)

  #Replace with mode as data RHS skewed
  df3['frac_fluid_intensity'].fillna(value=df.frac_fluid_intensity.median(), inplace=True)

  return df

df4 = fill_nan_1(df3)

<ipython-input-8-f49fe118dc67>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['bin_lateral_length'].fillna(value=df.bin_lateral_length.mean(), inplace=True)
<ipython-input-8-f49fe118dc67>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['gross_perforated_length'].fillna(value=df.gross_perforated_length.mean(), inplace=True)
<ipython-input-8-f49fe118dc67>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['proppant_intensity'].

In [9]:
df4.isna().sum()/len(df4[['OilPeakRate']])

surface_x                       0.000000
surface_y                       0.000000
gross_perforated_length         0.000000
total_fluid                     0.000000
true_vertical_depth             0.000000
ffs_frac_type                   0.258768
proppant_intensity              0.000000
frac_fluid_intensity            0.000000
proppant_to_frac_fluid_ratio    0.000000
frac_fluid_to_proppant_ratio    0.000000
bin_lateral_length              0.000000
relative_well_position          0.000000
batch_frac_classification       0.000000
well_family_relationship        0.000000
frac_seasoning                  0.250065
OilPeakRate                     0.000000
dtype: float64

### Using clustering to fillna for the 25% variables

In [10]:
df_cluster = df4.drop(["ffs_frac_type", "frac_seasoning"], axis=1)
df_cluster = df_cluster.select_dtypes(exclude='object')
#normalising the cluster df
scalar = MinMaxScaler()
df_cluster_scale = scalar.fit_transform(df_cluster)

In [11]:
#num_clusters = 6 as it is the number of objects
clust = KMeans(n_clusters=6)
clust.fit(df_cluster_scale)

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


KMeans(n_clusters=6)

In [12]:
clust.labels_

array([1, 1, 1, ..., 4, 4, 4], dtype=int32)

In [13]:
df_cluster["cluster"] = clust.labels_
df_cluster

,surface_x,surface_y,gross_perforated_length,total_fluid,true_vertical_depth,proppant_intensity,frac_fluid_intensity,proppant_to_frac_fluid_ratio,frac_fluid_to_proppant_ratio,bin_lateral_length,OilPeakRate,cluster
0,1.383494e+06,7.173292e+05,3963.582677,3.900855e+06,8712.598425,19.174572,570.930579,0.224015,5.586810,1.0,46.623023,1
1,1.380344e+06,7.119163e+05,3179.133858,5.610964e+05,8627.952756,35.544272,176.493480,0.201391,4.965455,1.0,59.750009,1
2,1.379916e+06,7.178450e+05,3810.039370,1.621424e+06,8801.181102,75.214821,425.566078,0.176741,5.658008,1.0,10.785716,1
3,1.383567e+06,7.066402e+05,3723.425197,4.377134e+05,8635.826772,28.704465,117.556659,0.244176,4.095414,1.0,123.797638,1
4,1.378963e+06,7.026240e+05,3811.023622,6.229802e+05,8664.370079,29.595648,163.467933,0.181049,5.523377,1.0,102.309540,1
...,...,...,...,...,...,...,...,...,...,...,...,...
29432,9.477118e+05,1.057502e+06,4101.377953,5.499711e+05,8877.952756,23.371936,134.094222,0.174295,5.737403,1.0,162.307168,4
29433,9.593475e+05,1.033564e+06,4512.795276,3.900855e+06,8479.330709,108.575936,570.930579,0.224015,5.586810,1.0,171.071455,4
29434,9.669162e+05,1.057804e+06,3660.433071,6.214390e+05,8640.748031,24.001844,169.771992,0.141377,7.073290,1.0,7.171827,4
29435,9.462629e+05,1.034169e+06,5930.118110,1.956479e+06,8389.763780,64.730101,329.922425,0.196198,5.096893,1.0,336.044270,4


In [14]:
merged_df = df4.merge(df_cluster, how='outer')
df_cols = merged_df[["ffs_frac_type", "cluster"]]
result = df_cols.groupby("cluster")["ffs_frac_type"].apply(lambda x: x.mode().iloc[0]).reset_index()
result_dict = dict(zip(result["cluster"].to_numpy(), result['ffs_frac_type'].to_numpy()))
result_dict

{0: 'Slickwater Only',
 1: 'Slickwater Crosslink Hybrid',
 2: 'Slickwater Only',
 3: 'Slickwater Crosslink Hybrid',
 4: 'Slickwater Crosslink Hybrid',
 5: 'Slickwater Only'}

In [15]:
#merged_df["ffs_frac_type"].fillna(result)
merged_df.ffs_frac_type = merged_df.ffs_frac_type.fillna(merged_df.cluster.map(result_dict))

In [16]:
df_cols1 = merged_df[["frac_seasoning", "cluster"]]
result1 = df_cols1.groupby("cluster")["frac_seasoning"].mean()
result_dict1 = dict(result1)

In [17]:
merged_df.frac_seasoning = merged_df.frac_seasoning.fillna(merged_df.cluster.map(result_dict1))

In [18]:
df5 = merged_df.drop(['cluster'], axis=1)

In [19]:
df5.isna().sum()/len(df4[['OilPeakRate']])

surface_x                       0.0
surface_y                       0.0
gross_perforated_length         0.0
total_fluid                     0.0
true_vertical_depth             0.0
ffs_frac_type                   0.0
proppant_intensity              0.0
frac_fluid_intensity            0.0
proppant_to_frac_fluid_ratio    0.0
frac_fluid_to_proppant_ratio    0.0
bin_lateral_length              0.0
relative_well_position          0.0
batch_frac_classification       0.0
well_family_relationship        0.0
frac_seasoning                  0.0
OilPeakRate                     0.0
dtype: float64

In [20]:
df5.ffs_frac_type.value_counts()

Slickwater Crosslink Hybrid    7680
Slickwater Only                6575
Slickwater Linear Hybrid       3078
Undefined                      1002
Crosslink Only                  894
Linear Only                      74
Name: ffs_frac_type, dtype: int64

In [21]:
#label encoding
def encode_cat_cols(df):
  label_encoder = LabelEncoder()
  df['relative_well_position']= label_encoder.fit_transform(df['relative_well_position'])
  df['well_family_relationship']= label_encoder.fit_transform(df['well_family_relationship'])
  df['batch_frac_classification']= label_encoder.fit_transform(df['batch_frac_classification'])
  frac_type_mapping = {
      'Undefined': 0,
      'Slickwater Only': 1,
      'Slickwater Linear Hybrid': 2,
      'Linear Only': 3,
      'Slickwater Crosslink Hybrid': 4,
      'Crosslink Only': 5
  }

  df['ffs_frac_type_enc'] = df['ffs_frac_type'].map(frac_type_mapping)
  df = df.drop('ffs_frac_type', axis = 1)
  return df

df6 = encode_cat_cols(df5)

In [22]:
pip install bayesian-optimization


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder
import xgboost as xgb
from bayes_opt import BayesianOptimization
from sklearn.preprocessing import OneHotEncoder
import numpy

df = df6
X = df.drop('OilPeakRate', axis=1)
y = df['OilPeakRate']



# Identify categorical columns
# categorical_cols = X.select_dtypes(include=['object']).columns

# # One-hot encode categorical columns
# X_encoded = pd.get_dummies(X, columns=categorical_cols)

# X_encoded = X_encoded.drop('well_family_relationship_Unknown', axis = 1)
# X_encoded = X_encoded.drop('well_family_relationship_Standalone Well', axis = 1)
# X_encoded = X_encoded.drop('ffs_frac_type_Linear Only', axis = 1)
# X_encoded = X_encoded.drop('batch_frac_classification_Unknown', axis = 1)


# X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, test_size=0.2, random_state=42)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# import re

# regex = re.compile(r"\[|\]|<", re.IGNORECASE)

# X_train.columns = [regex.sub("_", col) if any(x in str(col) for x in set(('[', ']', '<'))) else col for col in X_train.columns.values]

# X_test.columns = [regex.sub("_", col) if any(x in str(col) for x in set(('[', ']', '<'))) else col for col in X_test.columns.values]

print(X_train.columns)
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)

param_space = {
    'max_depth': (3, 10),
    'subsample': (0.3, 1),
    'colsample_bytree': (0.3, 1),
    'gamma': (0, 5),
    'min_child_weight': (1, 10),
    'eta': (0.01, 0.3)
}

def xgb_objective(max_depth, subsample, colsample_bytree, gamma, min_child_weight, eta):
    params = {
        'objective': 'reg:squarederror',
        'eval_metric': 'rmse',
        'max_depth': int(max_depth),
        'subsample': subsample,
        'colsample_bytree': colsample_bytree,
        'gamma': gamma,
        'min_child_weight': min_child_weight,
        'eta': eta
    }

    cv_results = xgb.cv(
        params,
        dtrain,
        num_boost_round=1000,
        nfold=5,
        metrics='rmse',
        early_stopping_rounds=50,
        seed=42
    )

    return -cv_results['test-rmse-mean'].iloc[-1]

xgb_bayesian_opt = BayesianOptimization(
    f=xgb_objective,
    pbounds=param_space,
    random_state=42,
)
xgb_bayesian_opt.maximize(init_points=5, n_iter=10)
best_params = xgb_bayesian_opt.max['params']
best_params['max_depth'] = int(best_params['max_depth'])
final_model = xgb.train(best_params, dtrain, num_boost_round=1000)
y_pred = final_model.predict(dtest)
mse = mean_squared_error(y_test, y_pred)
print(f'Optimized Mean Squared Error: {mse}')


Index(['surface_x', 'surface_y', 'gross_perforated_length', 'total_fluid',
       'true_vertical_depth', 'proppant_intensity', 'frac_fluid_intensity',
       'proppant_to_frac_fluid_ratio', 'frac_fluid_to_proppant_ratio',
       'bin_lateral_length', 'relative_well_position',
       'batch_frac_classification', 'well_family_relationship',
       'frac_seasoning', 'ffs_frac_type_enc'],
      dtype='object')
|   iter    |  target   | colsam... |    eta    |   gamma   | max_depth | min_ch... | subsample |
-------------------------------------------------------------------------------------------------
| 1         | -110.6    | 0.5622    | 0.2857    | 3.66      | 7.191     | 2.404     | 0.4092    |
| 2         | -105.2    | 0.3407    | 0.2612    | 3.006     | 7.957     | 1.185     | 0.9789    |
| 3         | -103.3    | 0.8827    | 0.07158   | 0.9091    | 4.284     | 3.738     | 0.6673    |
| 4         | -105.5    | 0.6024    | 0.09446   | 3.059     | 3.976     | 3.629     | 0.5565    |
| 

In [ ]:
rmse = mean_squared_error(y_test, y_pred, squared = False)
print(f'Optimized Mean Squared Error: {rmse}')

In [ ]:
xgb.plot_importance(final_model, importance_type='weight')
plt.show()

In [ ]:
submission = pd.DataFrame(submission, columns=['OilPeaks'])
submission.to_csv('/content/drive/MyDrive/Datathon 2024/Chevron/Test/submission_xgboost.csv')

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import seaborn as sns

df = df5

X = df.drop('OilPeakRate', axis=1)
y = df['OilPeakRate']
categorical_cols = X.select_dtypes(include=['object']).columns
X_encoded = pd.get_dummies(X, columns=categorical_cols)

X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, test_size=0.2, random_state=42)
n_components = 10
pca = PCA(n_components=n_components)
X_train_pca = pca.fit_transform(X_train)

pca_df = pd.DataFrame(data=X_train_pca, columns=[f'PC{i}' for i in range(1, n_components + 1)])
pca_df['Target'] = y_train.values

plt.figure(figsize=(10, 8))
sns.scatterplot(x='PC1', y='PC2', hue='Target', data=pca_df, palette='viridis', s=60)
plt.title('2D PCA Plot')
plt.xlabel('Principal Component 1 (PC1)')
plt.ylabel('Principal Component 2 (PC2)')
plt.legend(title='Target', loc='best')
plt.show()
